In [1]:
import sncosmo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sncosmo.salt2utils import BicubicInterpolator
from scipy.interpolate import (
    InterpolatedUnivariateSpline as Spline1d,
    RectBivariateSpline as Spline2d
)
import pickle as pk
from matplotlib.backends.backend_pdf import PdfPages
import os 
from matplotlib.offsetbox import AnchoredText
import astropy.constants as const
import astropy.units as u
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
import glob
sns.set_style("whitegrid", {'axes.grid':'False','xtick.bottom': True,'ytick.left': True})

params = {"text.usetex" : True,'font.size': 20, 'font.family':'serif', 'font.serif':'Computer Modern'}
plt.rcParams.update(params)

H_ERG_S = const.h.cgs.value
C_AA_PER_S = const.c.to(u.AA / u.s).value
HC_ERG_AA = H_ERG_S * C_AA_PER_S
SCALE_FACTOR = 1e-12

In [2]:
class ScalarFormatterClass1(ScalarFormatter):
    def _set_format(self):
        self.format = "%1.1f"

## Comentários

In [3]:
#

## Recuperando curvas de luz

In [4]:
path="./light_curves/Pantheon/Pantheon_PS1MD_TEXT/"

In [5]:
l=[]
with open(path+"Pantheon_PS1MD_TEXT.README",'r') as f:
    for line in f:
        l.append(line.split())

In [6]:
sne_pass_cut=[l[8:][i][0] for i in range(0,len(l[8:]))]

In [7]:
len(sne_pass_cut)

282

In [8]:
red_err={}

In [9]:
for f in glob.glob(path+"*.txt"):
    file=open(f)
    for line in file:
        if line.startswith("SNID:"):
            cid=str(line[6:-1].strip())
        if line.startswith("REDSHIFT_FINAL:"):
            try:
                red_err[cid]=float(line.split("+-")[1])
            except:
                red_err[cid]=float(line.split("+-")[1].split(" ")[1])

In [10]:
red_err

{'310260': 0.01,
 '580276': 0.001,
 '360139': 0.001,
 '350374': 0.001,
 '580300': 0.001,
 '110036': 0.001,
 '590097': 0.001,
 '350606': 0.001,
 '460037': 0.001,
 '520008': 0.001,
 '350050': 0.001,
 '330146': 0.001,
 '370189': 0.001,
 '040111': 0.001,
 '500511': 0.001,
 '160066': 0.001,
 '330143': 0.001,
 '000248': 0.001,
 '370563': 0.001,
 '500100': 0.01,
 '130451': 0.001,
 '080646': 0.001,
 '340316': 0.001,
 '530219': 0.01,
 '040166': 0.001,
 '180561': 0.001,
 '160099': 0.001,
 '040782': 0.001,
 '500038': 0.001,
 '140181': 0.001,
 '370394': 0.01,
 '090201': 0.001,
 '110716': 0.001,
 '560054': 0.001,
 '380318': 0.001,
 '050251': 0.001,
 '110567': 0.001,
 '590002': 0.01,
 '000202': 0.001,
 '440005': 0.001,
 '370308': 0.01,
 '120369': 0.001,
 '330083': 0.001,
 '110033': 0.001,
 '050088': 0.001,
 '050580': 0.001,
 '100358': 0.001,
 '470043': 0.01,
 '080735': 0.001,
 '550137': 0.001,
 '500057': 0.01,
 '530061': 0.001,
 '030006': 0.001,
 '091869': 0.001,
 '090275': 0.001,
 '440285': 0.01,
 

## Decodificando filtros

In [11]:
filters=["g","r","i","z","y"]

In [12]:
def path_filter(f):
    return "/home/cassia/SNANA/snroot/filters/PS1/Pantheon/PS1/"+f+"_filt_tonry.txt"

## Registrando filtros e criando sistema de magnitude composto

In [13]:
for f in filters:
    wave = pd.read_csv(path_filter(f),header=None,sep="\s+",comment="#")[0].values
    trans = pd.read_csv(path_filter(f),header=None,sep="\s+",comment="#")[1].values
    band = sncosmo.Bandpass(wave, trans, name='PS1_'+f,trim_level=0.05)
        
    sncosmo.registry.register(band)

In [14]:
ab=sncosmo.get_magsystem('ab')

In [15]:
magsystem = {"PS1_g":('ab',-0.023-0.0037),
             "PS1_r":('ab',-0.033-0.0066),
             "PS1_i":('ab',-0.024-0.0043),
             "PS1_z":('ab',-0.024+.008),
             "PS1_y":('ab',0.)}

## Criando modelo EXP

In [16]:
class EXP_LC(sncosmo.Source):
    
    _param_names = ['x0', 'x1', 'x2']
    param_names_latex = ['x_0', 'x_1', 'x_2']
    
    
    def __init__(self, phase, wave, flux0, flux1, flux2, name='EXP', version='2021'):

        self.name = name
        self.version = version
        self._phase = phase
        self._wave = wave
        
        self._model_flux0  = BicubicInterpolator(phase, wave, flux0.T)
        self._model_flux1  = BicubicInterpolator(phase, wave, flux1.T)
        self._model_flux2  = BicubicInterpolator(phase, wave, flux2.T)

        self._parameters = np.array([1, 0, 0])  # initial guess
        

    def _flux(self, phase, wave):
        
        x0, x1, x2= self._parameters
        #print('flux!')
        return x0 * (self._model_flux0(phase, wave) +  x1 * self._model_flux1(phase, wave) +  x2 * self._model_flux2(phase, wave))

In [17]:
gridx2 = np.linspace(-10, 50, 50+10+1, dtype=int)
gridy2 = np.linspace(3400, 8400, int((8400-3400)/10+1), dtype=int)

In [18]:
X, Y = np.meshgrid(gridx2,gridy2)

#### Templates FA

In [19]:
M_fa=[]
for i in range(3):
    M_fa.append(np.loadtxt("../fa/sk_varimax_M"+str(i)+".txt")*SCALE_FACTOR)

#### Templates PCA

In [20]:
M_pca=[]
for i in range(3):
    M_pca.append(np.loadtxt("../pca/M"+str(i)+"_pca.txt")*SCALE_FACTOR)

### Criando os modelos

In [21]:
exp_fa = EXP_LC(gridx2, gridy2, M_fa[0], M_fa[1], M_fa[2])
exp_pca = EXP_LC(gridx2, gridy2, M_pca[0], M_pca[1], M_pca[2])

In [22]:
sncosmo.register(exp_fa,name="exp_fa_varimax")
sncosmo.register(exp_pca,name="exp_pca")

## Escolha do modelo

In [23]:
modelo="exp_pca"
offset=8.065 # additive term from mb

## Ajustando curvas de luz

In [24]:
def lc_fit(sn):    

    meta, tables = sncosmo.read_snana_ascii(path+"PS1_PSc"+sn+".txt", default_tablename='OBS')
    
    dat=tables["OBS"]
    surv="PS1"

    dat.add_column([surv+"_"+f for f in dat["FLT"]], name='FILTER') 
    dat.remove_column("FLT")
    
    dat.remove_rows(np.where(dat["FLUXCAL"]<0)[0])
    dat.remove_rows(np.where(np.isnan(dat["MAG"]))[0])
    dat.remove_columns(["FIELD","MAG","MAGERR"])

    magsys=sncosmo.CompositeMagSystem(bands=magsystem,name="magsys")
    sncosmo.register(magsys,force=True)
    
    dat.remove_rows(np.where(dat["FILTER"]==surv+"_z")[0])

    if sn in ["500038","490008","350316","010026"]:
        dat.remove_rows(np.where(dat["FILTER"]==surv+"_g")[0])
    
    if sn in ["590194","550005","530086","520022","490007","470043","370566","370066",
              "070242","080064","000137","420100","370356","370333","360156","300105",
              "140181","110567","100405","100163"]:
        dat.remove_rows(np.where(dat["FILTER"]==surv+"_i")[0])

    
    dat["MAG"]=-2.5*np.log10(dat["FLUXCAL"])+27.5
    dat["MAGERR"]=abs(2.5*np.log10(np.e)*dat["FLUXCALERR"]/dat["FLUXCAL"])

    dat["FLUX"]=[magsys.band_mag_to_flux(dat["MAG"][i],dat["FILTER"][i]) for i in range(len(dat["MAG"]))]
    dat["FLUXERR"]=[abs(0.4*dat["FLUX"][i]*np.log(10.)*dat["MAGERR"][i]) for i in range(len(dat["MAG"]))]
  
    # ref de repouso: (t-t0)/(1+z), no SNANA fazem um corte de -15 a 45 no referencial de repouso. E também tem um corte no 
    # intervalo de comprimento de onda, eu apliquei esses cortes abaixo em phase_range e wave_range. Porém não tenho certeza
    # se o sncosmo corta no ref de repouso ou do observador. Alterando isso, os valores de x1 para 2001ah e 2001az mudam drasticamento
    # equivalendo a mesma mudança que ocorre no snana quando fazemos a mesma alteração no TREST_REJECT.    
    
    dat["ZP"]=[2.5*np.log10(magsys.zpbandflux(f)) for f in dat["FILTER"]]
    dat.add_column("magsys", name='ZPSYS')
    
    dust = sncosmo.F99Dust() 
    
    # para algumas sne como 2006bb o chi2 da minimizacao da Nan e não é possível ajustar a sn. A mesma sn no snana é ajustada pelo salt.
    # alterando intervalo de dias e comprimento de onda de atuação do ajustador, ainda não é possível evitar o nan já no primeiro passo.
    
    model = sncosmo.Model(source=modelo,effects=[dust],effect_names=['mw'],effect_frames=['obs'])
    model.set(mwebv=meta["MWEBV"],z=meta["REDSHIFT_FINAL"]) 
    try:
        result, fitted_model = sncosmo.fit_lc(dat, model, ['t0', 'x0', 'x1', 'x2'],modelcov=False,phase_range=(-10, 40.), wave_range=(3400.,7000.), verbose=False)
    except:
        print(f"SN {sn} fit returns NaN")
        result=None
        fitted_model=None
    
    return meta, surv, dat, result, fitted_model

In [25]:
pp = PdfPages(f"./lc_fit_plot_{modelo.upper()}_PS1_Pantheon.pdf")

In [26]:
colors1=["#440154"]
colors2=["#fde725","#440154"]
colors3=["#fde725","#21918c","#440154"]
colors4=["#fde725","#35b779","#31688e","#440154"]

In [27]:
def return_fit_data(sn,meta,res,model_name):

    return [meta["SNID"], meta["SURVEY"], meta["REDSHIFT_FINAL"], red_err[sn], res.parameters[1],
           res.errors["t0"], res.parameters[2], res.errors["x0"], res.parameters[3],
           res.errors["x1"], res.parameters[4], res.errors["x2"],
           res.covariance[1][2], res.covariance[1][3], res.covariance[2][3], 
           res.chisq/res.ndof, model_name]

In [28]:
def lc_fit_plot(sn, surv,table_data, result, fitted_model):
    z=round(result["parameters"][0],2)
    t0=round(result["parameters"][1],2)
    x1=round(result["parameters"][3],2)
    x1err=round(result["errors"]["x1"],2)
    x2=round(result["parameters"][4],2)
    x2err=round(result["errors"]["x2"],2)
    mwebv=result["parameters"][5]
    phase=np.linspace(t0-10.,t0+50.,100)

    if result.ndof>0:
        fil=[]
        for i,f in enumerate(magsystem.keys()): 
            if len(table_data[table_data["FILTER"]==f])==0:
                continue
            else:
                try:
                    fitted_model.bandflux(f, phase)
                    fil.append(f)
                    continue
                except:
                    continue
        fig, axs=plt.subplots(len(fil),1,figsize=(10,4*len(fil)),sharex=True)
        fig.subplots_adjust(hspace=0.1)
        if len(fil)==1:
            for i,f in enumerate(fil): 
                colors=eval("colors"+str(len(fil)))
                axs.plot(phase-t0,fitted_model.bandflux(f, phase)*10**(-0.4*table_data[table_data["FILTER"]==f]["ZP"][0]+11),linewidth=3,color=colors[i],label=f"{modelo.upper()} fit ($\chi^2/ndof$={round(result.chisq/result.ndof,2)})")
                axs.errorbar(table_data[table_data["FILTER"]==f]["MJD"]-t0,table_data[table_data["FILTER"]==f]["FLUXCAL"],yerr=table_data[table_data["FILTER"]==f]["FLUXCALERR"],label=f"Flux through {f} filter",ls="none", marker='o',markersize=8,color=colors[i])
                axs.set_ylabel(r"FLUX ($Z_{AB}$=27.5)")
                axs.set_xlim(-12,52)
                axs.set_title(f"{sn}({surv}),   z:{z:.2f},   x1={x1:.2f}$\pm${x1err:.2f},   x2={x2:.2f}$\pm${x2err:.2f}")
                formatter11=ScalarFormatterClass1()
                formatter11.set_scientific(True)
                formatter11.set_powerlimits((0,0))
                axs.yaxis.set_major_formatter(formatter11)
                axs.legend(loc=3)
                if i==len(fil)-1:
                    axs.set_xlabel(f"MJD-{t0}")
            #plt.show()
            plt.tight_layout()
            pp.savefig()
            plt.close(fig)
        else:
            for i,f in enumerate(fil): 
                colors=eval("colors"+str(len(fil)))
                axs[i].plot(phase-t0,fitted_model.bandflux(f, phase)*10**(-0.4*table_data[table_data["FILTER"]==f]["ZP"][0]+11),linewidth=3,color=colors[i],label=f"{modelo.upper()} fit ($\chi^2/ndof$={round(result.chisq/result.ndof,2)})")
                axs[i].errorbar(table_data[table_data["FILTER"]==f]["MJD"]-t0,table_data[table_data["FILTER"]==f]["FLUXCAL"],yerr=table_data[table_data["FILTER"]==f]["FLUXCALERR"],label=f"Flux through {f} filter",ls="none", marker='o',markersize=8,color=colors[i])
                axs[i].set_ylabel(r"FLUX ($Z_{AB}$=27.5)")
                axs[i].set_xlim(-12,52)
                axs[i].set_title(f"{sn}({surv}),   z:{z:.2f},   x1={x1:.2f}$\pm${x1err:.2f},   x2={x2:.2f}$\pm${x2err:.2f}")
                formatter11=ScalarFormatterClass1()
                formatter11.set_scientific(True)
                formatter11.set_powerlimits((0,0))
                axs[i].yaxis.set_major_formatter(formatter11)
                axs[i].legend(loc=3)
                if i==len(fil)-1:
                    axs[i].set_xlabel(f"MJD-{t0}")
            #plt.show()
            plt.tight_layout()
            pp.savefig()
            plt.close(fig)
    else:
        pass

In [29]:
all_fit_data=[]
f=open(f"./lc_failed_fits_{modelo.upper()}_PS1_Pantheon.txt","w")
for j,sn in enumerate(sne_pass_cut):
    print(sn, f"({j}/{len(sne_pass_cut)})")
    meta, surv, table_data, result, fitted_model=lc_fit(sn) 
    
    if result is None:
        f.write(f"SN {sn} fit returns NaN\n")
        continue
    else:
        if result.success and result.ndof>0:
            all_fit_data.append(return_fit_data(sn,meta,result,f"{modelo.upper()}"))
        elif result.success and result.ndof==0:
            f.write("ndof 0 for SN"+sn+"\n")
        elif not result.success:
            f.write("Unsuccessful fit of SN"+sn+"\n")

    lc_fit_plot(sn, surv, table_data, result, fitted_model)       

000006 (0/282)


/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: PS1_g(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


000011 (1/282)


/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: PS1_g(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


000034 (2/282)


...

/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: PS1_g(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


590194 (279/282)
520016 (280/282)


/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: PS1_g(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


380199 (281/282)


/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: PS1_g(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


In [30]:
pp.close()

In [31]:
f.close()

## Salvando dados dos ajustes

In [32]:
df=pd.DataFrame(all_fit_data)

In [33]:
df=pd.DataFrame(all_fit_data)
df.columns=("SNNAME", "SURVEY", "Z", "ZERR", "T0", "T0ERR", "X0", "X0ERR", "X1", "X1ERR", "X2/C", "X2/CERR", "COV_X0_X1", "COV_X0_X2/C", "COV_X1_X2/C", "CHISQ/NDOF", "MODEL")

In [34]:
df["mB"]=-2.5*np.log10(df["X0"])+offset
df["mBERR"]=np.abs(-2.5*np.log10(np.e)/df["X0"]*df["X0ERR"])
df["COV_mB_X1"]=-2.5*np.log10(np.e)/df["X0"]*df["COV_X0_X1"]
df["COV_mB_X2/C"]=-2.5*np.log10(np.e)/df["X0"]*df["COV_X0_X2/C"]

In [35]:
df.to_csv(f"./lc_fit_results_{modelo.upper()}_PS1_Pantheon.txt", sep=' ', index=False)